## Oil Prices

In [1]:
import time
import json
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


url = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html'
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
data = []

driver = webdriver.Chrome(options=opts)
driver.get(url)
time.sleep(2)
table = driver.find_element(By.TAG_NAME, 'tbody')
rows = table.find_elements(By.TAG_NAME, 'tr')

data_dir = Path('data')
if (data_dir / 'oilPrice.csv').exists():
    df_prices = pd.read_csv(data_dir / 'oilPrice.csv')
    last_saved_period = df_prices.iloc[0]
    last_period = rows[0].text.split()
    if last_saved_period[0] == last_period[0] and last_saved_period[1] == last_period[1]:
        print('There is no new prices')
    else:
        for row in rows:
            data.append(row.text.split())
        df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
        df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
        df.to_csv(data_dir / 'oilPrice.csv')

for row in rows:
    data.append(row.text.split())
df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
df.to_csv(data_dir / 'oilPrice.csv')

driver.quit()


KeyboardInterrupt: 

In [4]:
import os
os.chdir('..')

In [5]:
%pwd

'c:\\Users\\User\\Desktop\\DS\\Projects\\oil-optimization'

In [6]:
import time
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def scrape_table_rows(driver):
    table = driver.find_element(By.TAG_NAME, 'tbody')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    data = [row.text.split() for row in rows]
    
    df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
    df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
    df['Year'] = df['Year'].astype(int)

    return df

def update_csv(file_path, new_df):
    if file_path.exists():
        df_old = pd.read_csv(file_path)
        last_saved_year, last_saved_month = df_old.iloc[0,:2]
        last_new_year, last_new_month = new_df.iloc[0,:2]

        if (last_saved_year == last_new_year) and (last_saved_month == last_new_month):
            print('There are no new prices.')
            return
        
    
    new_df.to_csv(file_path,index=False)
    print('Data Updated')

if __name__ == '__main__':
    url = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html'
    data_dir = Path('data')
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir / 'oilPrice.csv'
    opts = Options()
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    opts.add_argument("--headless=new")
    driver = webdriver.Chrome(options=opts)
    driver.get(url)
    time.sleep(2)

    df = scrape_table_rows(driver)
    update_csv(file_path, df)

    driver.quit()


There are no new prices.


## Oil Indexes

In [19]:
import requests
from bs4 import BeautifulSoup
url1 = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/SectorReal/ix_PetroleoMinas.html'

r = requests.get(url1, verify='bce.crt')
soup = BeautifulSoup(r.text)

In [20]:
soup

<!DOCTYPE html>

<html lang="es">
<head>
<!-- ConfiguraciÃ³n del documento -->
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" https-equiv="Content-Type">
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>EstadÃ­sticas de PetrÃ³leo y Minas</title>
<!-- Metadatos -->
<meta content="PÃ¡gina WEB sobre el EstadÃ­sticas de PetrÃ³leo y Minas del Banco Central del Ecuador" name="description"/>
<meta content="EstadÃ­sticas de PetrÃ³leo y Minas, InformaciÃ³n EconÃ³mica, Banco Central del Ecuador, BCE" name="keywords"/>
<!-- Iconos PestaÃ±a -->
<link href="https://contenido.bce.fin.ec/documentos/informacioneconomica/componentes/img/favicon.ico" rel="icon"/>
<link href="https://contenido.bce.fin.ec/documentos/informacioneconomica/componentes/img/favicon.ico" rel="apple-touch-icon"/>
<!-- Iconos PestaÃ±a -->
<link href="https://contenido.bce.fin.ec/documentos/informacioneconomica/componentes/img/favicon.ico" rel="icon"/>
<link href="https://contenido.b

In [29]:
soup.find(name='section',attrs={'id':'section_CSPxls'}).find_all(name='a')[1].get('href')

'https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx'

In [5]:
import requests

r = requests.get('https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx', verify='../bce.crt')
with open('../data/oilFigures.xlsx', 'wb') as f:
    f.write(r.content)

In [136]:
import pandas as pd
import numpy as np
df = pd.read_excel('../data/oilFigures.xlsx',nrows=161)

In [137]:
df = df.iloc[:,1:]
idx = df[df.iloc[:,0] == 'PRODUCCIÓN DE PETRÓLEO CRUDO (Miles de barriles)'].index[0]
df = df.iloc[idx+1,1:]

In [ ]:
chunk_size = 12
for i, year in range(0,len())

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])

In [66]:
years = df.iloc[0].unique()[1:].astype(int)

In [160]:
data = pd.DataFrame()
for i,year in enumerate(years):
    chunk = df[i*12:i*12+12].to_frame()
    chunk['year'] = year
    data = pd.concat([data,chunk],axis=0)

In [166]:
data.reset_index(drop=True).rename(columns={3:'Total'})

,Total,year
0,16029.77266,2007
1,14202.41541,2007
2,14944.46022,2007
3,15061.70616,2007
4,15862.39327,2007
...,...,...
218,13996.89146,2025
219,14148.01696,2025
220,14461.57249,2025
221,14014.07711,2025


## PetroEcuador Reports

In [27]:
import tabula

data = tabula.read_pdf('https://www.eppetroecuador.ec/wp-content/uploads/downloads/2025/09/INFORME-ESTADISTICO-MENSUAL-AGOSTO-2025-2.pdf', pages='54')

In [28]:
data[0]

,E X P O R T A C I O N E S D E C R U D O O R I E N T E Y N A P O\rR E A L I Z A D A S P O R E P P E T R O E C U A D O R (a)\rAño 2025,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Meses,Crudo Oriente,Crudo Napo,Total Crudo,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Volumen\rBls,Precio\rUS$/bl,Valor\rUS$ FOB,Volumen\rBls,Precio\rUS$/bl,Valor\rUS$ FOB,Volumen\rBls,Precio\rUS$/bl,Valor\rUS$ FOB
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Enero,6.375.867,"69,77","444.818.561,47",4.250.806,"63,61","270.373.091,57",10.626.673,"67,30","715.191.653,04"
4,Febrero,5.241.934,"66,16","346.830.049,65",3.149.453,"62,60","197.165.486,04",8.391.387,"64,83","543.995.535,69"
5,Marzo,5.241.225,"63,71","333.940.587,44",3.809.427,"58,00","220.961.229,10",9.050.652,"61,31","554.901.816,54"
6,Abril,5.329.962,"58,06","309.480.139,31",3.890.849,"53,96","209.945.724,55",9.220.811,"56,33","519.425.863,85"
7,Mayo,5.312.803,"57,11","303.424.996,99",3.179.459,"51,64","164.171.432,29",8.492.262,"55,06","467.596.429,28"
8,Junio,8.727.698,"61,62","537.772.150,78",3.893.069,"58,08","226.104.257,12",12.620.768,"60,53","763.876.407,90"
9,Julio,683.554,"62,22","42.532.978,92",1.027.400,"55,25","56.765.925,08",1.710.953,"58,04","99.298.904,00"


## Energy Information Administration (EIA) Api

In [2]:
import json
import pandas as pd

In [11]:
with open('../response.json') as f:
    data = json.load(f)

In [20]:
df = pd.DataFrame(data['response']['data']).sort_values(by='period')

In [25]:
df[df['destinationName'].eq('United States')]

,period,originId,originName,originType,originTypeName,destinationId,destinationName,destinationType,destinationTypeName,gradeId,gradeName,quantity,quantity-units
281,2024-01,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,MED,Medium,33,thousand barrels
15,2024-01,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,2462,thousand barrels
192,2024-02,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,3795,thousand barrels
194,2024-03,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,3521,thousand barrels
276,2024-04,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,2996,thousand barrels
92,2024-05,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,5798,thousand barrels
228,2024-06,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,4576,thousand barrels
111,2024-07,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,5229,thousand barrels
76,2024-08,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,3541,thousand barrels
148,2024-09,CTY_EC,Ecuador,CTY,Country,US,United States,US,United States,HSO,Heavy Sour,3410,thousand barrels
